# Activity #1

You are a new Data Scientist working for Austin 3-1-1, and your manager has asked you to answer the 
following questions: 
1. Can you predict if a new 3-1-1 call is going to be successfully resolved?  (Note that a call is 
considered to be successfully resolved if it has a status of ‘resolved’ or ‘closed’) 
2. Can you predict the mean time to resolution? 
3. Are there any other interesting insights? 

Go ahead and impress your manager with your data mining and machine learning skills!  Use any tool of 
your choice to build your models and demonstrate your insights.   

# Part 1

## Import

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [59]:
df = pd.read_csv('/content/drive/MyDrive/Data Science for Engineers/austin_311.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Observe

In [60]:
df.head()

,Service Request (SR) Number,SR Description,Method Received,SR Status,SR Location,Street Number,Street Name,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate,(Latitude.Longitude),Council District,Created Date,Closed Date
0,19-00090956,Animal - Proper Care,Phone,Closed,"4609 RIBBECKE AVE, AUSTIN, TX 78721",4609,RIBBECKE,AUSTIN,78721.0,TRAVIS,30.280581,-97.685318,"(30.28058076, -97.68531793)",1.0,2019-03-14 03:02:15,2020-11-23 01:41:21
1,20-00135805,Tree Issue ROW,Mobile Device,Closed,"3521 WEST AVE, AUSTIN, TX 78705",3521,WEST,AUSTIN,78705.0,TRAVIS,30.302857,-97.742312,"(30.30285681, -97.74231249)",9.0,2020-04-07 07:06:32,2020-11-23 12:02:05
2,20-00052942,Pothole Repair,Web,Closed,"7900 E BEN WHITE BLVD WB, AUSTIN, TX",7900,BEN WHITE BLVD WB,AUSTIN,78741.0,TRAVIS,30.219838,-97.688440,"(30.21983764, -97.68844011)",3.0,2020-02-12 01:55:04,2020-02-12 12:57:35
3,19-00467964,ARR Dead Animal Collection,Phone,Closed,"7100 METROPOLIS DR, AUSTIN, TX 78744",7100,METROPOLIS,AUSTIN,78744.0,TRAVIS,30.206036,-97.702592,"(30.20603574, -97.70259216)",2.0,2019-12-10 02:17:04,2019-12-11 09:57:54
4,19-00468033,Austin Code - Request Code Officer,Phone,Closed,"2401 GROVE BLVD, AUSTIN, TX 78741",2401,GROVE,AUSTIN,78741.0,TRAVIS,30.221664,-97.707177,"(30.22166434, -97.70717698)",3.0,2019-12-10 02:51:24,2019-12-10 03:00:05


In [61]:
df.columns

Index(['Service Request (SR) Number', 'SR Description', 'Method Received',
       'SR Status', 'SR Location', 'Street Number', 'Street Name', 'City',
       'Zip Code', 'County', 'Latitude Coordinate', 'Longitude Coordinate',
       '(Latitude.Longitude)', 'Council District', 'Created Date',
       'Closed Date'],
      dtype='object')

In [62]:
df.describe()

,Zip Code,Latitude Coordinate,Longitude Coordinate,Council District
count,899447.000000,903257.000000,903257.000000,858106.000000
mean,78729.543079,30.226383,-97.763444,5.506708
std,26.508744,1.310668,0.406121,3.025361
min,78610.000000,3.442387,-105.983195,1.000000
25%,78705.000000,30.232492,-97.773683,3.000000
50%,78739.000000,30.279274,-97.741555,5.000000
75%,78749.000000,30.345623,-97.705675,9.000000
max,78759.000000,30.623016,-97.375982,10.000000


In [63]:
df.isna().sum()

Service Request (SR) Number         0
SR Description                      0
Method Received                     0
SR Status                           0
SR Location                      9044
Street Number                  178320
Street Name                      9059
City                            12110
Zip Code                        12854
County                          17726
Latitude Coordinate              9044
Longitude Coordinate             9044
(Latitude.Longitude)             9044
Council District                54195
Created Date                        0
Closed Date                      1960
dtype: int64

In [64]:
df.dtypes

Service Request (SR) Number     object
SR Description                  object
Method Received                 object
SR Status                       object
SR Location                     object
Street Number                   object
Street Name                     object
City                            object
Zip Code                       float64
County                          object
Latitude Coordinate            float64
Longitude Coordinate           float64
(Latitude.Longitude)            object
Council District               float64
Created Date                    object
Closed Date                     object
dtype: object

In [65]:
df.shape

(912301, 16)

## Preprocess

### Impute mean in numerical columns

In [66]:
cols_null = [col for col in df.columns if df[col].isna().any() and df[col].dtype in ['int64', 'float64']]
print([cols_null])

[['Zip Code', 'Latitude Coordinate', 'Longitude Coordinate', 'Council District']]


In [67]:
# Use simple imputer
imputer = SimpleImputer(missing_values=np.NaN, strategy='mean')
df[cols_null] = imputer.fit_transform(df[cols_null])

df[cols_null].isna().sum()

Zip Code                0
Latitude Coordinate     0
Longitude Coordinate    0
Council District        0
dtype: int64

### Impute most frequent value in categorical columns

Check for columns with many unique values

In [68]:
[col for col in df.columns if df[col].dtype in ['object'] and df[col].nunique() > 10]

['Service Request (SR) Number',
 'SR Description',
 'Method Received',
 'SR Status',
 'SR Location',
 'Street Number',
 'Street Name',
 'City',
 '(Latitude.Longitude)',
 'Created Date',
 'Closed Date']

Check unique values for certain features to see if they can be simplified

In [69]:
df['SR Status'].unique()

array(['Closed', 'Duplicate (closed)', 'Resolved', 'Open',
       'CancelledTesting', 'Duplicate (open)', 'New', 'Work In Progress',
       'Closed -Incomplete', 'TO BE DELETED', 'Incomplete',
       'Closed -Incomplete Information', 'Transferred'], dtype=object)

In [70]:
# Clean up SR Status into two unique values

df['SR Status'] = df['SR Status'].replace(['Closed', 'Duplicate (closed)', 'Resolved', 'CancelledTesting', 'Closed -Incomplete',
                                           'TO BE DELETED', 'Closed -Incomplete Information', 'Transferred'], 
                                          1)
df['SR Status'] = df['SR Status'].replace(['Open', 'Duplicate (open)', 'New', 'Work In Progress', 'Incomplete'], 
                                          0)

df['SR Status'].unique()

array([1, 0])

In [71]:
df['City'].nunique()

134

In [72]:
df['SR Description'].nunique()

143

In [73]:
df['Street Name'].nunique()

38077

In [74]:
df['County'].nunique()

6

In [75]:
df['Method Received'].nunique()

18

Get all categorical columns

In [76]:
cols_cat = [col for col in df.columns if df[col].dtype in ['object']]
cols_cat

['Service Request (SR) Number',
 'SR Description',
 'Method Received',
 'SR Location',
 'Street Number',
 'Street Name',
 'City',
 'County',
 '(Latitude.Longitude)',
 'Created Date',
 'Closed Date']

Impute values

In [77]:
imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
df[cols_cat] = imputer.fit_transform(df[cols_cat])

df[cols_cat].isna().sum()

Service Request (SR) Number    0
SR Description                 0
Method Received                0
SR Location                    0
Street Number                  0
Street Name                    0
City                           0
County                         0
(Latitude.Longitude)           0
Created Date                   0
Closed Date                    0
dtype: int64

### Create dummy variables

In [78]:
cols_to_drop = [col for col in df.columns if df[col].dtype in ['object'] and df[col].nunique() > 1000]

x = df.drop(cols_to_drop, axis=1).drop('SR Status', axis=1)
#x = pd.get_dummies(x, drop_first=True)
y = df['SR Status']

x.head()

,SR Description,Method Received,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate,Council District
0,Animal - Proper Care,Phone,AUSTIN,78721.0,TRAVIS,30.280581,-97.685318,1.0
1,Tree Issue ROW,Mobile Device,AUSTIN,78705.0,TRAVIS,30.302857,-97.742312,9.0
2,Pothole Repair,Web,AUSTIN,78741.0,TRAVIS,30.219838,-97.688440,3.0
3,ARR Dead Animal Collection,Phone,AUSTIN,78744.0,TRAVIS,30.206036,-97.702592,2.0
4,Austin Code - Request Code Officer,Phone,AUSTIN,78741.0,TRAVIS,30.221664,-97.707177,3.0


In [79]:
cols = [col for col in x.columns if x[col].dtype in ['object']]
labelencoder = LabelEncoder()

for col in cols:
  x[col] = labelencoder.fit_transform(x[col])

x.head()

,SR Description,Method Received,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate,Council District
0,12,12,2,78721.0,4,30.280581,-97.685318,1.0
1,130,9,2,78705.0,4,30.302857,-97.742312,9.0
2,92,17,2,78741.0,4,30.219838,-97.688440,3.0
3,2,12,2,78744.0,4,30.206036,-97.702592,2.0
4,22,12,2,78741.0,4,30.221664,-97.707177,3.0


### Split data

In [80]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

## Build model

Logistic regression

In [81]:
log = LogisticRegression().fit(x_train, y_train)
pred = log.predict(x_test)
log.score(x_test, y_test)

0.9958073232087953

# Part 2

## Select data

Note: This is a very literal way of calculating the mean of the time to resolution. I had to look some stuff up because I'm not 100% what was expected for a "prediction" of this.

In [89]:
a = df.loc[df['SR Status'] == 1].loc[:, ['Created Date', 'Closed Date']]
a.head()

,Created Date,Closed Date
0,2019-03-14 03:02:15,2020-11-23 01:41:21
1,2020-04-07 07:06:32,2020-11-23 12:02:05
2,2020-02-12 01:55:04,2020-02-12 12:57:35
3,2019-12-10 02:17:04,2019-12-11 09:57:54
4,2019-12-10 02:51:24,2019-12-10 03:00:05


In [108]:
a['Created Date'] =  pd.to_datetime(a['Created Date'], format='%Y-%m-%d %H:%M:%S')
a['Closed Date'] =  pd.to_datetime(a['Closed Date'], format='%Y-%m-%d %H:%M:%S')

b = a['Closed Date']-a['Created Date']

In [110]:
b.pipe(lambda d: (lambda m: m + (d - m).mean())(d.min()))

Timedelta('32 days 21:08:24.041615344')

Here is the average time to resolution: 32 days, 21 hours, 8 minutes, and 24 seconds. Obviously, this isn't a very pretty solution but I don't see why this wouldn't qualify.

# Part 3

In [116]:
log.score(x_test, y_test)

0.9958073232087953

The score is very high. That's good, but let's see how legit it is.

In [111]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[     0,    765],
       [     0, 181696]])

No true positives

No false negatives

Lots of false positives

Lots of true negatives

(Something is off)

In [112]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred)

0.5

Quoted from a Google search: 

"This ROC curve has an AUC of 0.5, meaning it ranks a random positive example higher than a random negative example 50% of the time. As such, the corresponding classification model is basically worthless, as its predictive ability is no better than random guessing."

(The validity of my model isn't looking too good)

In [115]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       765
           1       1.00      1.00      1.00    181696

    accuracy                           1.00    182461
   macro avg       0.50      0.50      0.50    182461
weighted avg       0.99      1.00      0.99    182461



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report pretty much sums it up. My model's score looks very good but once you dig deeper it gets ugly. It's ugly ROC score suggests that it isn't much better than just random guessing.

It seems that it is totally perfect at predicting Closed cases (represented by 1) and totally imperfect at predicting Open cases (represented by 0). This is probably due to how I set up my model. Maybe I can find out why in class.